<a href="https://colab.research.google.com/github/Rasha-Abd-El-Khalik/Tashkhees/blob/main/medicaltest_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import files
# uploaded = files.upload()  # pick your .zip file


In [ ]:
# import zipfile
# with zipfile.ZipFile("marbert_finetuned_lora.zip", 'r') as zip_ref:
#     zip_ref.extractall(".")


In [ ]:
import zipfile
import os

# Unzip only if folder does not already exist
if not os.path.exists("marbert_finetuned_lora"):
    with zipfile.ZipFile("marbert_finetuned_lora.zip", 'r') as zip_ref:
        zip_ref.extractall("marbert_finetuned_lora")


In [ ]:
import pandas as pd
df=pd.read_csv("medical_test.csv")
df.head()

,Category,Question,Answer
0,الغدد الصماء,عملت تحليل للغدة الدرقية وطلعت 9.41 وعندي سنة ...,إذا كان الtsh هو المرتفع فهذا دليل على وجود خم...
1,الغدد الصماء,هل يوثر البلوغ المبكر علي الطول\nمع العلم اصبح...,نعم البلوغ يوقف الطول تدريجيا حتى يتوقف بشكل ن...
2,جراحة العظام,السلام ع انا مصاب بالجنف اجريت العملية ومرة عل...,قم باعادة الاشعة وكذلك الرنين لتقيم الحالة
3,الغدد الصماء,انا عملت عملية استصال الغدة الكرضية الاثنتين ه...,الحمدالله على السلامه و قدامك العافيه يارب\nال...
4,جراحة عامة,وجدت حباية تحت الابط الايمن و تحولت من حباية ظ...,الغالب أنها غدة لمفية، وقد تكون تكلست أيضاً، ر...


In [ ]:
category_counts = df['Category'].value_counts()

print(category_counts)

Category
الغدد الصماء            1048
جراحة عامة              1017
أمراض الجهاز التنفسي    1003
مرض السكري               964
جراحة العظام             810
Name: count, dtype: int64


In [ ]:
# Check nulls and datatypes
nulls_info = pd.DataFrame({
    "Null_Count": df.isnull().sum(),
    "DataType": df.dtypes
})

print(nulls_info)


          Null_Count DataType
Category           0   object
Question           0   object
Answer             0   object


In [ ]:
duplicates_count = df.duplicated().sum()
print("Total duplicate rows:", duplicates_count)

Total duplicate rows: 0


In [ ]:
df = df.drop(columns=["Answer"])

In [ ]:
import unicodedata

df['Question'] = df['Question'].apply(lambda text: unicodedata.normalize('NFKC', text))


In [ ]:
# pip install camel-tools


In [ ]:
from camel_tools.utils.normalize import normalize_alef_maksura_ar, normalize_alef_ar, normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
import re

def camel_clean(text):
    text = dediac_ar(str(text))
    text = normalize_alef_ar(text)
    text = normalize_alef_maksura_ar(text)
    text = normalize_teh_marbuta_ar(text)
    en2ar = str.maketrans('0123456789', '٠١٢٣٤٥٦٧٨٩')
    text = text.translate(en2ar)
    text = re.sub(r'[^\u0600-\u06FF\s.,;:!?()\'"-]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

df['Question'] = df['Question'].apply(camel_clean)


In [ ]:

import re

def extract_non_arabic(text):
    """
    Extract all characters that are NOT Arabic letters.
    """
    if not isinstance(text, str):
        return ""
    return ''.join(re.findall(r'[^\u0600-\u06FF]', text))

all_text = ''.join(df['Question'].astype(str).tolist())
non_arabic_chars = set(extract_non_arabic(all_text))

print(non_arabic_chars)
print(f"\nTotal unique non-Arabic characters found: {len(non_arabic_chars)}")


{'.', ';', ',', "'", ' ', ')', '(', '-', ':', '!', '"', '?'}

Total unique non-Arabic characters found: 12


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Category'])


NUM_LABELS = df['Label'].nunique()
print("Labels:", label_encoder.classes_)
print("Number of classes:", NUM_LABELS)


Labels: ['أمراض الجهاز التنفسي' 'الغدد الصماء' 'جراحة العظام' 'جراحة عامة'
 'مرض السكري']
Number of classes: 5


In [ ]:
df = df.rename(columns={'Label': 'labels'})

In [ ]:
df['labels'] = df['labels'].astype('int64')

In [ ]:
category_counts = df['Category'].value_counts()

In [ ]:
# !pip install transformers datasets peft accelerate bitsandbytes


In [ ]:
# !pip install --upgrade transformers
# !pip install --upgrade peft


In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model
import pandas as pd

In [ ]:
test_dataset = Dataset.from_pandas(df)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./marbert_finetuned_lora")

# Load base MARBERT
base_model = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=5)

# Load LoRA fine-tuned weights
model = PeftModel.from_pretrained(base_model, "./marbert_finetuned_lora")
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(100000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (defau

In [ ]:
df.columns

Index(['Category', 'Question', 'labels'], dtype='object')

In [ ]:
df["labels"]

,labels
0,1
1,1
2,2
3,1
4,3
...,...
4837,0
4838,4
4839,4
4840,0


In [ ]:
df.shape

(4842, 3)

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm  # progress bar


df['label'] = df['labels']

# Rename Question → text
df = df.rename(columns={"Question": "text"})

# Convert to HuggingFace Dataset
test_dataset = Dataset.from_pandas(df)

# Tokenizer
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# DataLoader
test_loader = DataLoader(test_dataset, batch_size=16)

# --- Evaluation ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_preds, all_labels = [], []

model.eval()
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating", unit="batch"):
        outputs = model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device)
        )
        preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
        labels = batch["label"].cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels)

# Metrics
acc = accuracy_score(all_labels, all_preds)
f1  = f1_score(all_labels, all_preds, average="weighted")

print("✅ Test Accuracy:", acc)
print("✅ Test F1 Score:", f1)

# Classification Report (labels كأرقام)
unique_labels = sorted(set(all_labels) | set(all_preds))  # نتأكد نغطي كل الكلاسات
class_names = [str(i) for i in unique_labels]

print("\n📊 Classification Report:\n",
      classification_report(all_labels, all_preds, labels=unique_labels, target_names=class_names))


Map:   0%|          | 0/4842 [00:00<?, ? examples/s]

Evaluating: 100%|██████████| 303/303 [00:31<00:00,  9.53batch/s]

✅ Test Accuracy: 0.86575795125981
✅ Test F1 Score: 0.8644716132699657

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92      1003
           1       0.90      0.75      0.82      1048
           2       0.84      0.86      0.85       810
           3       0.84      0.85      0.84      1017
           4       0.83      0.96      0.89       964

    accuracy                           0.87      4842
   macro avg       0.87      0.87      0.86      4842
weighted avg       0.87      0.87      0.86      4842



In [ ]:
# !pip install gradio gTTS SpeechRecognition pydub camel-tools torch transformers peft
# !sudo apt-get install -y ffmpeg


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.5/556.5 kB 39.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# !pip install -U transformers peft


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.4
    Uninstalling transformers-4.43.4:
      Successfully uninstalled transformers-4.43.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
camel-tools 1.5.6 requires transformers<4.44.0,>=4.0, but you have transformers 4.56.1 which is incompatible.


In [ ]:
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import re, torch
from camel_tools.utils.normalize import normalize_alef_maksura_ar, normalize_alef_ar, normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# --------- تحميل الموديل والـ Tokenizer ---------
tokenizer = AutoTokenizer.from_pretrained("./marbert_finetuned_lora")

# Load base MARBERT
base_model = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=5)

# Load LoRA fine-tuned weights
model = PeftModel.from_pretrained(base_model, "./marbert_finetuned_lora")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ---------------- تنظيف النص ----------------
def camel_clean(text):
    text = dediac_ar(str(text))
    text = normalize_alef_ar(text)
    text = normalize_alef_maksura_ar(text)
    text = normalize_teh_marbuta_ar(text)
    en2ar = str.maketrans('0123456789', '٠١٢٣٤٥٦٧٨٩')
    text = text.translate(en2ar)
    text = re.sub(r'[^\u0600-\u06FF\s.,;:!?()\'"-]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

# ---------------- Speech to Text ----------------
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language="ar-EG")
        except:
            text = "❌ ماقدرتش أفهم الصوت"
    return text

def predict_text(text):
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=-1).item()
    return pred

def map_class(pred):
    mapping = {
        0: "أمراض الجهاز التنفسي",
        1: "الغدد الصماء",
        2: "جراحة العظام",
        3: "جراحة عامة",
        4: "مرض السكري"
    }
    return mapping.get(pred, str(pred))

def text_to_speech(text):
    if text.strip() == "":
        return None
    tts = gTTS(text=text, lang="ar")
    tts.save("output.mp3")
    return "output.mp3"

def process(audio_file):

    text = speech_to_text(audio_file)


    clean = camel_clean(text)


    pred = predict_text(clean)


    mapped = map_class(pred)


    speech_file = text_to_speech(mapped)

    return text, clean, str(pred), mapped, speech_file

# ---------------- GUI ----------------
with gr.Blocks() as demo:
    gr.Markdown("## 🎤 تحويل الكلام → نص → كلاس → صوت")

    with gr.Row():
        audio_input = gr.Audio(sources=["microphone"], type="filepath", label="سجل صوتك هنا")

    with gr.Row():
        text_output = gr.Textbox(label="النص الأصلي")
        clean_output = gr.Textbox(label="النص بعد التنظيف")

    with gr.Row():
        pred_output = gr.Textbox(label="Prediction (ID)")
        mapped_output = gr.Textbox(label="Mapped Class")

    with gr.Row():
        audio_output = gr.Audio(label="النص محوّل لصوت")

    audio_input.change(fn=process, inputs=audio_input,
                       outputs=[text_output, clean_output, pred_output, mapped_output, audio_output])

demo.launch()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://55c7be7aeba30beb0d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# df.head(10)

,Category,text,labels,label
0,الغدد الصماء,عملت تحليل للغده الدرقيه وطلعت ٩.٤١ وعندي سنه ...,1,1
1,الغدد الصماء,هل يوثر البلوغ المبكر علي الطول مع العلم اصبح ...,1,1
2,جراحة العظام,السلام ع انا مصاب بالجنف اجريت العمليه ومره عل...,2,2
3,الغدد الصماء,انا عملت عمليه استصال الغده الكرضيه الاثنتين ه...,1,1
4,جراحة عامة,وجدت حبايه تحت الابط الايمن و تحولت من حبايه ظ...,3,3
5,الغدد الصماء,دواء كالسيبون هل يفيد بعد استئصال الغده الدرقي...,1,1
6,جراحة العظام,السلام عليكم ابلغ من العمر ٢٠ سنه قمت بعمليه ف...,2,2
7,مرض السكري,السلام عليكم انا فتاه مصابه بالسكري منذ ١٩ عام...,4,4
8,مرض السكري,دكتور عندي والدتي مريضه سكري والان عندها التها...,4,4
9,أمراض الجهاز التنفسي,سعله من اسبوع بدات بالبدايه اخذت بس ما فادني ب...,0,0


In [ ]:
import importlib

packages = [
    "gradio",
    "torch",
    "transformers",
    "tokenizers",
    "peft",
    "gtts",
    "speech_recognition",
    "camel_tools"
]

for pkg in packages:
    try:
        module = importlib.import_module(pkg)
        print(f"{pkg} version: {module.__version__}")
    except ModuleNotFoundError:
        print(f"{pkg} is not installed")
    except AttributeError:
        print(f"{pkg} version info not available")


gradio version: 5.44.1
torch version: 2.8.0+cu126
transformers version: 4.56.1
tokenizers version: 0.22.0
peft version: 0.17.1
gtts version: 2.5.4
speech_recognition version: 3.14.3
camel_tools version: 1.5.6
